In [1]:
!pip install h5py

In [3]:
import h5py
import numpy as np
import pandas as pd

In [4]:
dataset = h5py.File('../data/2017.h5', 'r')

In [5]:
dataset

<HDF5 file "2017.h5" (mode r)>

In [6]:
dataset.keys()

<KeysViewHDF5 ['fields']>

In [7]:
dataset["fields"]

<HDF5 dataset "fields": shape (1460, 21, 721, 1440), type "<f4">

In [40]:
#!cp -r /home/jupyter/extreme_weather_pred/FourCastNet/utils .
!cp -r /home/jupyter/extreme_weather_pred/FourCastNet/networks .
#!mv utils utilities

In [38]:
!ls

automl_3_hor_sel_fea_Dallas_tabular_forecasting_batch.ipynb
data_prep_era5.ipynb
data_prep_visual_crossing_weather.ipynb
filter_us.ipynb
run_afnonet_inference.ipynb
utilities


In [45]:
import os
import sys
import time
import numpy as np
import argparse
#sys.path.append("/home/jupyter/extreme_weather_pred/notebooks")
#sys.path.append("../FourCastNet")

#sys.path.append(os.path.dirname(os.path.realpath(__file__)) + '/../')
from numpy.core.numeric import False_
import h5py
import torch
import torchvision
from torchvision.utils import save_image
import torch.nn as nn
import torch.cuda.amp as amp
import torch.distributed as dist
from collections import OrderedDict
from torch.nn.parallel import DistributedDataParallel
import logging
from utilities import logging_utils
from utilities.weighted_acc_rmse import weighted_rmse_torch_channels, weighted_acc_torch_channels, unweighted_acc_torch_channels, weighted_acc_masked_torch_channels
logging_utils.config_logger()
from utilities.YParams import YParams
from utilities.data_loader_multifiles import get_data_loader
from networks.afnonet import AFNONet
import wandb
import matplotlib.pyplot as plt
import glob
from datetime import datetime


In [46]:
#!pip install wandb
#!pip install timm
#!pip install einops

In [13]:
!ls ../FourCastNet/utils/

YParams.py		   date_time_to_hours.py  weighted_acc_rmse.py
darcy_loss.py		   img_utils.py
data_loader_multifiles.py  logging_utils.py


In [48]:

fld = "z500" # diff flds have diff decor times and hence differnt ics
if fld == "z500" or fld == "2m_temperature" or fld == "t850":
    DECORRELATION_TIME = 36 # 9 days (36) for z500, 2 (8 steps) days for u10, v10
else:
    DECORRELATION_TIME = 8 # 9 days (36) for z500, 2 (8 steps) days for u10, v10
idxes = {"u10":0, "z500":14, "2m_temperature":2, "v10":1, "t850":5}

def gaussian_perturb(x, level=0.01, device=0):
    noise = level * torch.randn(x.shape).to(device, dtype=torch.float)
    return (x + noise)

def load_model(model, params, checkpoint_file):
    model.zero_grad()
    checkpoint_fname = checkpoint_file
    checkpoint = torch.load(checkpoint_fname)
    try:
        new_state_dict = OrderedDict()
        for key, val in checkpoint['model_state'].items():
            name = key[7:]
            if name != 'ged':
                new_state_dict[name] = val  
        model.load_state_dict(new_state_dict)
    except:
        model.load_state_dict(checkpoint['model_state'])
    model.eval()
    return model

def downsample(x, scale=0.125):
    return torch.nn.functional.interpolate(x, scale_factor=scale, mode='bilinear')


In [53]:
parser = argparse.ArgumentParser()
parser.add_argument("--run_num", default='00', type=str)
parser.add_argument("--yaml_config", default='./config/AFNO.yaml', type=str)
parser.add_argument("--config", default='full_field', type=str)
parser.add_argument("--use_daily_climatology", action='store_true')
parser.add_argument("--vis", action='store_true')
parser.add_argument("--override_dir", default=None, type = str, help = 'Path to store inference outputs; must also set --weights arg')
parser.add_argument("--interp", default=0, type=float)
parser.add_argument("--weights", default=None, type=str, help = 'Path to model weights, for use with override_dir option')

_StoreAction(option_strings=['--weights'], dest='weights', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, required=False, help='Path to model weights, for use with override_dir option', metavar=None)

In [54]:
parser

ArgumentParser(prog='ipykernel_launcher.py', usage=None, description=None, formatter_class=<class 'argparse.HelpFormatter'>, conflict_handler='error', add_help=True)

In [55]:
#args = parser.parse_args()
params = YParams(os.path.abspath(args.yaml_config), args.config)

NameError: name 'args' is not defined

In [ ]:
args = parser.parse_args()
params = YParams(os.path.abspath(args.yaml_config), args.config)
params['world_size'] = 1
params['interp'] = args.interp
params['use_daily_climatology'] = args.use_daily_climatology
params['global_batch_size'] = params.batch_size

torch.cuda.set_device(0)
    torch.backends.cudnn.benchmark = True
    vis = args.vis